### Data preprocessing

In [3]:
import mne
import numpy as np
print(mne.__version__)
print(np.__version__)
import sys
sys.path.append('../src/utils/data_preprocessing')
sys.path.append('../src/model')
sys.path.append('../src/data')

import temp_utils as utils
from  svm import train_svm_model

1.6.0
1.26.3


In [ ]:
file_path = r'../src/data/test_data/16/16.edf'
raw = utils.read_edf(file_path)
raw = utils.rename_edf(raw)
with mne.utils.use_log_level('WARNING'):
        raw.filter(1, 60, fir_design='firwin')

picks = utils.obtain_picks(raw)
info = utils.obtain_eeg_info(raw, picks)

In [ ]:
utils.obtain_multi_topomap(raw, picks, info, '16', fig_size=128, is_energy=True, is_test=False)

In [ ]:
from PIL import Image
import os

# 文件夹路径
main_path = r'D:\Research\EEG\EEG_DL_Classifier\tests'
eeg_file_name = '1'
folder_path = os.path.join(main_path, eeg_file_name)

output_path = r'D:\Research\EEG\EEG_DL_Classifier\tests\eeg.gif'


# 读取所有图像文件
images = []
for i in range(3600):
    file_name = f'{eeg_file_name}_{i}.png'  # 假设文件命名方式为 "0.png", "1.png", ..., "3599.png"
    file_path = os.path.join(folder_path, file_name)
    if os.path.exists(file_path):
        img = Image.open(file_path)
        images.append(img)




In [ ]:
gif_path = output_path
images[0].save(gif_path, save_all=True, append_images=images[0:], optimize=False, duration=100, loop=1)

In [ ]:
egg_filename = '1'
X_train, X_test, y_train, y_test = utils.prepare_data(egg_filename, is_test = False)

### Data Distribution

In [ ]:
import pandas as pd


# figure out whether the number of 0 is more than 1 in the label_df
def which_is_more(label_df):
    label_df = label_df['label'].values
    if (sum(label_df == 0) / (sum(label_df == 1)+sum(label_df == 0)) ) > 0.8:
            return 0
    elif (sum(label_df == 1) / (sum(label_df == 1)+sum(label_df == 0)) ) > 0.8:
            return 1

index = 33
normal_index, patient_index = [], []
for i in range(1,index + 1):
    eeg_file_name = str(i)
    with pd.ExcelFile(rf'../src/data/EEG_DATASET/{eeg_file_name}/{eeg_file_name}.xlsx') as xls:
        label_df = pd.read_excel(xls, 'Sheet1',header=None)
        label_df.rename(columns={0: 'label'}, inplace=True)
    
    if which_is_more(label_df) == 0:
        normal_index.append(i)
    else:
        patient_index.append(i)


print(f'病人: {patient_index}')
print(f'正常人: {normal_index}')

       


### Model Trainning

In [ ]:
eeg_file_names = ['1','16']
X_train, X_test, y_train, y_test = utils.prepare_data_from_multi_file(eeg_file_names, handle_data_imbalance=False, is_test = False)

In [ ]:
train_svm_model(X_train, y_train, X_test, y_test)

In [1]:
import torch
import pytorch_lightning as pl
import sys
sys.path.append('../src/data')
from data import EEG_Dataset, EEG_DataModule

def test_eeg_data_module():
    eeg_file_names = ['1']  # 指定 EEG 文件名列表

    # 初始化数据模块
    data_module = EEG_DataModule(eeg_file_names=eeg_file_names, test_mode = True, batch_size=2)  # 您可以调整 batch_size

    # 获取数据加载器
    train_loader = data_module.train_dataloader()
    val_loader = data_module.val_dataloader()
    test_loader = data_module.test_dataloader()

    # 遍历训练数据加载器进行测试
    for batch in train_loader:
        images, labels = batch
        print("Train batch - Images shape:", images.shape, "Labels shape:", labels.shape)
        break  # 只查看第一个 batch

    # 同样，您可以遍历验证和测试数据加载器
    for batch in val_loader:
        images, labels = batch
        print("Validation batch - Images shape:", images.shape, "Labels shape:", labels.shape)
        break

    for batch in test_loader:
        images, labels = batch
        print("Test batch - Images shape:", images.shape, "Labels shape:", labels.shape)
        break

# 运行测试函数
test_eeg_data_module()


Train batch - Images shape: torch.Size([2, 16384]) Labels shape: torch.Size([2])
Validation batch - Images shape: torch.Size([2, 16384]) Labels shape: torch.Size([2])
Test batch - Images shape: torch.Size([2, 16384]) Labels shape: torch.Size([2])
